In [1]:
import requests
import numpy as np
import pandas as pd
from io import StringIO

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pickle
import xgboost as xgb

In [2]:
url = "https://github.com/AndresLDF/ML-Zoomcamp-Midterm-Project/raw/main/DataSet/Concrete_Data_Yeh.csv"

In [3]:
response = requests.get(url)

In [4]:
if response.status_code==200:
  file = StringIO(response.text)
  df = pd.read_csv(file)


In [5]:
#Set all the columns labels to lower case
df.columns = df.columns.str.lower()

In [16]:
df.iloc[15]

cement              380.00
slag                  0.00
flyash                0.00
water               228.00
superplasticizer      0.00
coarseaggregate     932.00
fineaggregate       670.00
age                  90.00
csmpa                52.91
Name: 15, dtype: float64

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=4)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=4)
df_full_train.shape, df_train.shape, df_val.shape, df_test.shape

((824, 9), (618, 9), (206, 9), (206, 9))

In [7]:
y_fulltrain = df_full_train["csmpa"].values
y_train = df_train["csmpa"].values
y_val = df_val["csmpa"].values
y_test = df_test["csmpa"].values

In [8]:
del df_full_train["csmpa"]
del df_train["csmpa"]
del df_val["csmpa"]
del df_test["csmpa"]

In [9]:
X_fulltrain = df_full_train.values
X_train = df_train.values
X_val = df_val.values
X_test = df_test.values


In [10]:
features = df_train.columns.to_list()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)


In [11]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [12]:
features = df_full_train.columns.to_list()
dfulltrain = xgb.DMatrix(X_fulltrain, label=y_fulltrain, feature_names=features)
dtest = xgb.DMatrix(X_test, label=y_test, feature_names=features)

watchlist = [(dfulltrain, 'fulltrain'), (dtest, 'test')]

depth = 9
child_weight = 10

xgb_params = {
        'eta': 0.05,
        'max_depth': depth,
        'min_child_weight': child_weight,

        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'nthread': 8,

        'seed': 1,
        'verbosity': 0,
        }
evals_result = {}
model = xgb.train(xgb_params, dfulltrain, num_boost_round=200,
                      verbose_eval=0,
                      evals=watchlist,
                      evals_result=evals_result)
test_rmse = evals_result["test"]["rmse"][-1]

In [14]:

# Specify the filename to save the model parameters
model_filename = 'xgb_model.pkl'

# Save the model using pickle
with open(model_filename, 'wb') as model_file:
    pickle.dump(model, model_file)